In [25]:
from undetective_selenium_na import *
import random,re
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys 
from urllib import parse

In [26]:
##쿠팡 url에서 검색어,상품아이디 추출하는 함수.
def coupang_url_extract(url):
    global serch_keyword,serch_product_id
    serch_keyword = re.search(r"q=([^&]*)", url).group(1)
    serch_keyword=parse.unquote(serch_keyword)
    serch_keyword=serch_keyword.replace("+", " ")
    serch_product_id = int(re.search(r"products/(\d+)", url).group(1))
    print(f"검색키워드:{serch_keyword},상품아이디{serch_product_id}")
    return str(serch_keyword),int(serch_product_id)

def chrome_start(pc_name="박경희"):
    global driver, pcname
    pcname=pc_name
    driver=make_driver()
    driver.maximize_window()
    driver.get("https://www.coupang.com/")
    driver.implicitly_wait(10)

def Ac1_insert():
    chrome_start()
    time.sleep(random.uniform(1,2))
    element_typer("#headerSearchKeyword",serch_keyword)
    time.sleep(random.uniform(1,2))
    #엔터치기.
    driver.find_element(By.CSS_SELECTOR,"#headerSearchKeyword").send_keys(Keys.ENTER)

#가격필터 설정하기.
def price_filter(min_price,max_price):
    scroll_CssVer(".search-filter-options.search-price-filter")
    elements=driver.find_elements(By.CSS_SELECTOR,".param-pricerange")
    elements[0].send_keys(min_price)
    elements[1].send_keys(max_price)
    rand_click_cssVer(".btn-price-submit")


#가짜 페이지를 클릭하는 함수를 만들어야 함.
def fake_product(stat_handle=0,work_page=None,work_num=2,work_time=20):
    switch_handle(stat_handle)
    #해당페이지에서 작업할 경우.
    if work_page != None:
        select_page(work_page)
        time.sleep(random.uniform(1,3))
    #현재페이지의 상품을 수집을한다.
    Product_explore()
    #작업할 갯수를 설정한다.
    work_indexs=random.sample(Pure_Product_indexs,work_num)
    print(f"작업할 갯수는{work_num}, 작업할 인덱스는{work_indexs}")
    #상품 속성 불러오기.
    elements=driver.find_elements(By.CSS_SELECTOR,".search-product")
    for count in work_indexs:
        scroll_click_El(elements[count])
        time.sleep(3)
        pageview(start_handle=stat_handle+1,scroll_t=work_time)
        time.sleep(1)
        driver.close()
        switch_handle(stat_handle)
        time.sleep(1)

def Ac2_faketime(work_page=None,work_num=2,work_time=20):
    fake_product(stat_handle=0,work_page=work_page,work_num=work_num,work_time=work_time)


def Ac3_price_filter(minprice=None,maxprice=None):
    if minprice != None and minprice > 0 :
        price_filter(minprice,maxprice)


def Product_explore():
    global Pure_Product_ids,Pure_Product_indexs
    html=driver.page_source
    soup=BeautifulSoup(html,"html.parser")

    items=soup.select(".search-product")
    Pure_Product_ids=[]
    Pure_Product_indexs=[]
    for item in items:
        product_id=int(item.attrs["data-product-id"])
        #순수상품만 남긴다.
        if item.select_one(".search-product-wrap") !=None: #베스트아이템 아닌경우.
            if item.select_one(".ad-badge")==None: #광고상품이 아닌 경우.
                Pure_Product_ids.append(product_id)
                Pure_Product_indexs.append(int(items.index(item)))
            
def select_page(target_page=2):
    btns=driver.find_elements(By.CSS_SELECTOR,".btn-page>a")

    for btn in btns:
        if int(btn.text)==target_page:
            print(f"{btn.text}해당페이지로 이동")
            scroll_click_El(btn)
            break

#타겟을 서치하는 함수 만들어야 한다.
def target_serch(start_handle=0,start_page=2):
    global target_informations
    #상품 데이터를 수집한다.
    switch_handle(start_handle)
    while True:
        Product_explore() #홈페이지의 상품아이디 다 수집한다.
        #######현재 페이지에 내가 찾는 상품이 있는경우.#######
        if serch_product_id in Pure_Product_ids:
            print("해당페이지에 상품이 존재한다.")

            #타겟 상품의 정보 추가하기(페이지,인덱스)
                #현재 페이지 넘버 찾아내기.
            btns=driver.find_elements(By.CSS_SELECTOR,".btn-page>a")
            for btn in btns:
                if len(btn.get_attribute("class")) >=1:
                    target_page=int(btn.text)
                    print(f"타겟 페이지는{target_page}")
                #타겟인덱스 찾아내기.(중요함)
            target_index=Pure_Product_indexs[Pure_Product_ids.index(serch_product_id)] #같은 인덱스에 공통정보를 담고있다.
                #타겟정보를 찐으로 담는다.
            target_informations={"target_page":target_page,"target_index":target_index}
            print(f"타겟페이지는 {target_page}, 타겟상품의 인덱스는{target_index}")
            break
        else: 
            print("해당페에지에 상품이 존재하지 않는다.")
            select_page(start_page)
            start_page=start_page+1

#페이지 스크롤 하는 함수 만들어야 됨.
def pageview(start_handle=1,scroll_t=30,speed="super"):
    switch_handle(start_handle)
    cur_time=time.time()
   
    #상품 더보기 클릭하기.<*상페의 상품더보기가 있든 없든, css걸린다. style속성값으로 유무를 구분한다.*>
    element=driver.find_element(By.CSS_SELECTOR,".product-detail-seemore")
    try:
        if element.get_attribute("style") == 'display: block;':
            print("상품더보기 상품 버튼이 존재한다.")
            scroll_click_Css("#productDetail > div.product-detail-seemore > div",speed="super")
    except:
        print("상세페이지 더보기 없음.")
    #오류방지용(동적페이지라 밑에 속성을 css 다 안불러옴,)
    scroll_CssVer(".sdp-review__title",speed=speed)
    
    #시간초 채울떄까지 위아래 반복으로 이동한다.
    while True:
        if time.time() - cur_time >= scroll_t:
            break
        #아래로 이동한다.
        scroll_CssVer(".sdp-review__title")
        if time.time() - cur_time >= scroll_t:
            break
        #위로 이동한다.(쿠팡은 클래스로 집는게 좋다.)
        scroll_CssVer(".prod-buy-btn",speed=speed)

   
def cur_page():
    btns=driver.find_elements(By.CSS_SELECTOR,".btn-page>a")
    for btn in btns:
        if len(btn.get_attribute("class")) >=1:
            page=int(btn.text)
            print(f"타겟 페이지는{page}")
    return page

#타겟을 클릭하고 ,페이지 뷰까지 한번에 처리한다.
def target_go(start_handle=0,page_view_time=30):
    switch_handle(start_handle)
    target_page=target_informations["target_page"]
    target_index=target_informations["target_index"]

    if cur_page() != target_page:
        select_page(target_page)
        time.sleep(random.uniform(1,2))

    #상품 속성 불러오기.
    elements=driver.find_elements(By.CSS_SELECTOR,".search-product")
    scroll_click_El(elements[target_index])
    time.sleep(random.uniform(1,3))
    pageview(start_handle=start_handle+1,scroll_t=page_view_time)

def Ac4_targetgo(page_view_time):
    target_serch()
    target_go(page_view_time=page_view_time)

#결제전 사전 셋팅.
    #구매수량 입력하는 함수. 완성됨.
def purchase_quantity(num=int):
    element=driver.find_element(By.CSS_SELECTOR,".prod-quantity__input")
    scroll_ElVer(element)
    
    #기존의 데이터 지우기.
    element.send_keys(Keys.CONTROL,"a")
    element.send_keys(Keys.DELETE)
    element.send_keys(num)

    time.sleep(random.uniform(1,3))
    #구매옵션을 설정하는 함수다.
def general_payoption(num1=int,num2=int):
    print("일반옵션 메뉴입니다..")
    select_option=[]
    select_option.append(num1)
    select_option.append(num2)

        #해당페이지에 구매옵션이 있는 경우다. #오류(.prod-option__item") 의외의 css는 하위메뉴 선택후 그 다음이 안됨
    main_elements=driver.find_elements(By.CSS_SELECTOR,".prod-option__item")
    for main in main_elements:
        try: #메인메뉴가 엘리먼트중에, 짝퉁으로 속성에 들어가는 경우가 있다.
            time.sleep(random.uniform(1,3))
            scroll_click_El(main)
            time.sleep(1)
            # #하위메뉴 선택하는 구간.
            sub_elements=driver.find_elements(By.CSS_SELECTOR,".prod-option__list")
                #상위메뉴와 인덱스로 연결한다.
            sub_elements2=sub_elements[main_elements.index(main)] #0번째 작업때, 0번 하위메뉴를 불러온다.
                #하위메뉴 선택지 선택지 하나하나다.
            sub_elements3=sub_elements2.find_elements(By.CSS_SELECTOR,".prod-option-dropdown-item")
                #하위메뉴를 클릭한다.
            sub_elements3[select_option[main_elements.index(main)]-1].click() #sub_elements3 인덱스가 하위메뉴 선택지다.
            time.sleep(random.uniform(1,3))
        except:
            pass

def image_payopion(num1=int,num2=int):
    #상위메뉴 사이즈 클릭하고 선택하기. 없는 경우도 대비한다.
    print("의류옵션메뉴입니다.")
    try:    
        scroll_click_Css(".Dropdown-Select__Label__Container")
        time.sleep(random.uniform(1,3))
            #하위메뉴를 클릭한다.
        element=driver.find_elements(By.CSS_SELECTOR,".Dropdown-Select__Dropdown__Item")
        print(len(element))
        rand_click_elVer(element[num1-1])
        
    except:
        print("상위메뉴가 없다.")
    
    #2번쨰 메뉴 작업코드다.
    try:
        #2번째 메뉴가 텍스트 옵션인경우 
        if exist_element_check(".Text-Select__Item__Border"):
            subEl=driver.find_elements(By.CSS_SELECTOR,".Text-Select__Item__Border")
            rand_click_elVer(subEl[num2-1])
            
        #2번째 메뉴가 이미지 옵션인경우 
        elif exist_element_check(".Image-Select__Item__Img__Status"):
            imageEls=driver.find_elements(By.CSS_SELECTOR,".Image-Select__Item__Img__Status")
            rand_click_elVer(imageEls[num2-1])
    except:
        print("서브메뉴가 없다.")

#메인메뉴가 하나인경우다.".Image-Select__Item"
def payoption(num1=int,num2=int):
    #의류 옵션이 사각박스인경우,
    if exist_element_check(".Dropdown-Select__Label__Container"):
        image_payopion(num1,num2)
        #선택이미지 박스만 있는 경우다.
    elif exist_element_check(".Image-Select__Item"):
        image_payopion(num1,num2)
        #텍스트 박스만 있는 경우다.
    elif exist_element_check(".Text-Select__Item__Border"):
        image_payopion(num1,num2)
    #일반적인 옵션이 경우(사각박스 선택지.)
    elif exist_element_check(".prod-option__selected.multiple"):
        general_payoption(num1,num2)

def making_screen_folder():
    global file_path
    file_path="//vmware-host/Shared Folders/vmware공유파일" 
    
    #실전, 테스트 환경을 구분하는 파트다.
    if os.path.exists(file_path):
        file_path="//vmware-host/Shared Folders/vmware공유파일/스크린샷폴더"
    else:    
        print("테스트환경이다.")
        file_path=filepathfix()
    
    #2폴더를 생성하는 파트다.
    if not os.path.exists(f"{file_path}"):
        os.mkdir(file_path)

    if not os.path.exists(f"{file_path}/쿠팡"):
        os.mkdir(f"{file_path}/쿠팡")

    if not os.path.exists(f"{file_path}/쿠팡/{coupang_product_name}"):
        os.mkdir(f"{file_path}/쿠팡/{coupang_product_name}")


#찜작업하는 함수.
def jjimwork():
    scroll_click_Css(".prod-favorite-btn")
    time.sleep(random.uniform(1,2))
    element=driver.find_element(By.CSS_SELECTOR,".prod-atf")
    element.screenshot(f"{file_path}/쿠팡/{coupang_product_name}/({pcname})찜.png")


#상품명을 수집하는 모듈이다.
def collect_productname():
    element=driver.find_element(By.CSS_SELECTOR,".prod-buy-header__title")
    product_name=element.text
    return product_name

#쿠팡 리뷰 수집하기.
#전체적인 형태는 리스트다.[(인덱스, 내용),(인덱스, 내용),(인덱스, 내용),(인덱스, 내용)]
def copang_review_crolling():
    switch_handle(-1)
    html=driver.page_source
    soup=BeautifulSoup(html,"html.parser")
    reviews=soup.select(".sdp-review__article__list__review__content.js_reviewArticleContent")
    review_list=[]
    for index, review in enumerate(reviews,1):
        print(f"{index}번째 리뷰")
        print("======================================")
        print(review.text.strip())
        묶음=(f"{index}번째",review.text.strip())
        review_list.append(묶음)
    return review_list


#구매수량,구매옵션, 찜작업을 해준다.(전파트중에 제일중요하다)
def Ac5_payoptions(moq=1,jjim=bool,optionmenus1=None,optionmenus2=None):
    global coupang_product_name,review_collects,coupang_product_price
    ####엑셀작업을 위한 정보수집####
    coupang_product_name=collect_productname()
    review_collects=copang_review_crolling() #전체적인 형태는 리스트다.[(인덱스, 내용),(인덱스, 내용),(인덱스, 내용),(인덱스, 내용)]
    coupang_product_price=int(driver.find_element(By.CSS_SELECTOR,".total-price").text[:-1].replace(",","")) #원본은 15,900원이다 ("원"단위제거, "," 제거했다)
    #스크린샷 저장을 위한 폴더만들기.
    making_screen_folder()
    #구입수량이 한개 이상일떄 작동한다.
    if moq > 1:
        purchase_quantity(num=moq)
    #찜 작업의 여부를 체크한다.
    if jjim:
        jjimwork()
    #옵션메뉴의 작동여부.
    if optionmenus1 > 0 or optionmenus2 > 0:
        payoption(num1=optionmenus1,num2=optionmenus2)
    
    #결제 방식의 2가지
def showpingback():
    scroll_click_Css(".prod-cart-btn")
    time.sleep(0.5)
    #장바구니로 이동하기.
    scroll_click_Css(".prod-order-notifier-link")
    time.sleep(1)
    #두번째 창에서 구매버튼 누르기.
    scroll_click_Css(".goPayment")

#바로 구매 버튼으로 구매하기.
def directpaybtn():
    scroll_click_Css(".prod-buy-btn")

#장바구니, 바로구매 둘중하나 클릭을 한다.
def Ac6_payways(jangbaguni=bool):
    #2중 1택이다. 장바구니라면, 그것을 실행 아니면, 다른것 실행하면 된다.
    if jangbaguni:
        showpingback()
        time.sleep(3)
    else:
        directpaybtn()
        time.sleep(3)

##쿠팡 포인트사용##
def pointuse():
    scroll_click_Css(".insert-cash-toggle")
    time.sleep(1)
    #모두사용버튼 누르기
    rand_click_cssVer(".cashAllUsing")
    time.sleep(1)
    #캐시적용버튼 누르기.
    rand_click_cssVer(".apply active")

#배송메세지 입력하기.
def basongmsg(msg=str):
    #메세지 변경버튼 누르기
    switch_handle(1)
    scroll_click_Css("td>.delivery-request-message__popup-list-button")
    time.sleep(random.uniform(1,2))
    switch_handle(2)
        #선택옵션,기타버튼을 누른다.
    rand_click_cssVer("body > div > div > div.content-wrapper > div.content-body.content-body--fixed > div > form > div.preference-required.__AA01_OTHER_PLACE > label > span")
    time.sleep(random.uniform(1,2))
        #기존의 모든 내용을 지운다.
    element=driver.find_element(By.CSS_SELECTOR,"#__AA01_OTHER_PLACE")
    ActionChains(driver).click(element).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()

        #배송메세지를 입력한다.
    element_typer("#__AA01_OTHER_PLACE",msg)

        #확인버튼 누른다.
    time.sleep(random.uniform(1,2))
    rand_click_cssVer(".delivery-preferences__save-button")

#결제 비번을 누른다.
def paymentpassworlds():
    psw_nums=[1,2,3,4,5]
    for psw_num in psw_nums:
        img_filepath=filepathfix()
        while True:
            try:
                exist_image=pyautogui.locateCenterOnScreen(f"{img_filepath}/C_Payment_password/{psw_num}.png")
            except:
                exist_image=pyautogui.locateCenterOnScreen("./C_Payment_password/1.png")
            if exist_image != None:
                break
        pyautogui.moveTo(exist_image)
        pyautogui.moveRel(random.randint(-20,20),random.randint(-5,5))
        pyautogui.click()
        time.sleep(random.randint(1,2))


#두번재 결제창을 누르는 함수.
def realpaymentbtns():
    switch_handle(1)
    scroll_click_Css("#paymentBtn")

#마지막 결제완료 후,스크린샷 남기는 것이다.
def lastscreenshot():
    element=driver.find_element(By.CSS_SELECTOR,".prod-atf")
    element.screenshot(f"{file_path}/쿠팡/{coupang_product_name}/({pcname})결제완료.png")
    

def Ac7_lastoption(point=bool,msg=None):
    #배송메세지 기능.
    if len(msg)>=1:
        basongmsg(msg)
    #포인트사용시
    if point:
        pointuse()

    realpaymentbtns()
    time.sleep(2)
    paymentpassworlds()

#구분을 위한 무늬 함수다.
def output():
    global coupang_product_name,review_collects,coupang_product_price
    coupang_product_name #ac5에 존재함.
    review_collects #ac5에 존재함.
    coupang_product_price #ac5에 존재함.
    ###추가 보강사항이 필요하다###
    #마지막에 포인트를 사용했다면 사용포인트.
    

#잘 작동하는지 테스트가 필요하다.
def coupang_start(url,minprice=None or int,maxprice=None or int,page_view_time=120,moq=int,jjim=bool,optionmenus1=None or int,optionmenus2=None or int,jangbaguni=bool,point=bool,msg=None or str):
    try:
        coupang_url_extract(url=url)
        Ac1_insert()
        Ac2_faketime(work_page=None,work_num=2,work_time=20)
        Ac3_price_filter(minprice=minprice,maxprice=maxprice)
        Ac4_targetgo(page_view_time)
        Ac5_payoptions(moq=moq,jjim=jjim,optionmenus1=optionmenus1,optionmenus2=optionmenus2)
        Ac6_payways(jangbaguni=jangbaguni)
        Ac7_lastoption(point=point,msg=msg)
        output()
        print("가구매작업이 끝났습니다.")
    except Exception as e:
        print("문제가 발생되었습니다.")
        # print(f"오류 발생 위치: {e.__traceback__.tb_frame.f_code.co_name}")
        e.traceback.print_tb(e.__traceback__)
 


In [27]:
#----------------------------------------------사용방법-------------------------------------------------
if __name__ == "__main__":
    url="https://www.coupang.com/vp/products/6991549817?itemId=17113539517&vendorItemId=84287173261&q=%EA%B2%BD%EC%B6%94%EB%B2%A0%EA%B0%9C&itemsCount=36&searchId=7f07ee5bcf2d4dc88b0adc5c37ab3cbd&rank=3&isAddedCart="
    minprice=13000
    maxprice=20000
    page_view_time= 30
    moq= 2
    jjim=True
    optionmenus1=2
    optionmenus2=2
    jangbaguni=False
    point=False
    msg="그냥 잘 배송해주면 좋구요."
    coupang_start(url=url,minprice=minprice,maxprice=maxprice,page_view_time=page_view_time,moq=moq,jjim=jjim,optionmenus1=optionmenus1,optionmenus2=optionmenus2,jangbaguni=jangbaguni,point=point,msg=msg)

검색키워드:경추베개,상품아이디6991549817


c:\Users\black\AppData\Local\Programs\Python\Python310\lib\site-packages\undetected_chromedriver\__init__.py:338: UserWarning: using ChromeOptions.user_data_dir might stop working in future versions.use uc.Chrome(user_data_dir='/xyz/some/data') in case you need existing profile folder
  warnings.warn(


작업할 갯수는2, 작업할 인덱스는[25, 22]
인덱스는:start
506 395 0.018134714063544152
인덱스는:121
509 378 0.2892546421746161
인덱스는:122
509 377 0.02386767213663073
인덱스는:123
509 376 0.02387472906564382
인덱스는:124
510 374 0.04853095610876962
인덱스는:125
679 414 2.4787636167064413
인덱스는:126
679 415 0.020054654040847684
인덱스는:127
679 416 0.015280300099184536
인덱스는:128
679 416 0.013370148765350774
인덱스는:129
679 416 0.02387495670851521
인덱스는:130
984 481 2.529130967858714
인덱스는:131
984 481 0.018145413278499483
인덱스는:132
984 481 0.01432579353944613
인덱스는:133
984 481 0.01433307811133061
인덱스는:134
984 481 0.02100164838583022
인덱스는:135
984 475 0.3046512677810809
인덱스는:136
984 475 0.02866023750800508
인덱스는:137
884 498 0.37435984021526525
인덱스는:138
882 499 0.02864931065017836
인덱스는:139
882 499 0.034381358151779375
인덱스는:140
606 372 1.5074581512738077
인덱스는:141
608 378 0.02674052517357294
인덱스는:142
609 380 0.022920450148776805
인덱스는:143
727 403 0.45746269155916774
인덱스는:144
729 404 0.017181801003905477
인덱스는:145
730 404 0.017199101862131118
인덱스는:1

AttributeError: 'NoSuchElementException' object has no attribute 'traceback'

In [3]:
url="https://www.coupang.com/vp/products/6991549817?itemId=17113539517&vendorItemId=84287173261&q=%EA%B2%BD%EC%B6%94%EB%B2%A0%EA%B0%9C&itemsCount=36&searchId=7f07ee5bcf2d4dc88b0adc5c37ab3cbd&rank=3&isAddedCart="
coupang_url_extract(url=url)

검색키워드:경추베개,상품아이디6991549817


('경추베개', 6991549817)

In [4]:
Ac1_insert()

c:\Users\black\AppData\Local\Programs\Python\Python310\lib\site-packages\undetected_chromedriver\__init__.py:338: UserWarning: using ChromeOptions.user_data_dir might stop working in future versions.use uc.Chrome(user_data_dir='/xyz/some/data') in case you need existing profile folder
  warnings.warn(


In [ ]:
switch_handle(-1)
scroll_click_Css("#productDetail > div.product-detail-seemore > div",speed="fast")

In [ ]:
switch_handle(-1)
scroll_CssVer(".sdp-review__title",speed="super")

In [26]:
scroll_time(20,"up","super")

In [ ]:
Ac2_faketime()

In [ ]:
Ac3_price_filter(13000,20000)

In [ ]:
review_collects=copang_review_crolling()

In [17]:
print(review_collects)

[('1번째 리뷰', '코멧 홈 입체슬립 베게, 차콜그레이 \n\n####간단요약###\n솜이 야박하게 들어있지않아요. 빵빵한 정도도 아니고 딱 보통양의 솜.\n푹신한 솜베게입니다. \n베개는 불편함없고, 목통증 이런거 없이 잘 자고 일어나지네요.\n높이는 보통정도인것 같아요. \n베게를 반 접어진 상태로 포장되어있었는데, 좀 별로였습니다. \n————————————————————————-\n\n• 구매일: 2022년 09월\n\n•제조년월: 2021년 8월 \n\n\n\n✔️솜, 푹신함\n솜은 너무 야박하지않게, 너무 풍족하지도 않게 그냥 적당하게 들어있습니다. \n아주 빵빵한 베개의 느낌은 아니고, 푹신한 솜 베개느낌입니다. \n\n솜이 부족한 느낌은 안들었습니다. \n바느질이나, 마감처리는 잘 되어있네요. \n테두리부분이 흰색으로 처리되어있어, 보기에는 좀 더 고급져보이고 좋네요. \n디자인은 약간 가성비 호텔베게 느낌이 들어요. \n\n푹신함은 푹신하고, 베개를 바꾸면 몇일은 불편할수도있는데, 그런거 없이 잘 사용중입니다. \n\n\n✔️색상, 커버, 재질\n색상은 차콜그레이를 구매했습니다. \n판매사진이랑 색상은 유사하더라구요. 밝은 회색느낌이 아닌 차콜 느낌 딱 어두운 진그레이 느낌이라 좋네요. 제가 원하던 색상이라 맘에 쏙 드네요. \n\n\n베게 커버안쪽으로 솜이 들어있는데, 솜부분엔 또 지퍼가 따로 되어있더라구요. \n충전재 부분에 속지퍼, 겉지퍼 따로 구분되어있네요. \n지퍼는 잘 열리고, 잘 닫혀요. \n\n사용하다보면 솜이 꺼지거나 하면 추가해도 될듯합니다. \n솜이 좀 죽는다싶으면 빈 페트병으로 탁탁 때려주면 좀 살아난다고 합니다. \n\n\n소재는 겉면, 충전재 모두 폴리에스터 100%네요. \n베개는 그냥 커버 안씌우고 사용해도 될듯해요. \n커버가 꺼칠하지않고 그냥 베고자도 전혀 불편함없더라구요. \n\n촉감도 괜찮은 편이네요. \n\n\n✔️세탁 \n세탁은 전체 한번에 세탁 가능하다고하네요. \n솜까지 세탁이 가능한가봐요. \n세

In [7]:
Ac4_targetgo(20)

해당페이지에 상품이 존재한다.
타겟 페이지는1
타겟페이지는 1, 타겟상품의 인덱스는2
타겟 페이지는1
인덱스는:start
1049 337 0.539840642069993
상품더보기 상품 버튼이 존재한다.
인덱스는:start
490 388 0.01971429118259754
인덱스는:716
567 439 0.17025616095036897
인덱스는:717
577 440 0.008033919343888711
인덱스는:718
584 441 0.00968641855209075
인덱스는:719
587 442 0.006458381986569326
인덱스는:720
590 443 0.008507106457065558
인덱스는:721
651 480 0.43539756169594945
인덱스는:722
652 478 0.010217198783678116
인덱스는:723
653 477 0.5015567906492107
인덱스는:724
653 476 0.005919135951384868
인덱스는:725
655 475 0.00587449807787293
인덱스는:726
1072 401 0.5709547025573654
인덱스는:727
1070 401 0.015241524948800668
인덱스는:728
1067 401 0.009148326944669532
인덱스는:729
1064 401 0.010039160368636245
인덱스는:730
1044 401 0.03600583231694054
인덱스는:731
953 396 0.1850493831177782
인덱스는:732
953 402 0.008596382204089436
인덱스는:733
953 407 0.006997499752002313
인덱스는:734
953 408 0.006456201400794318
인덱스는:735
953 410 0.008551744330577498
인덱스는:736
953 414 0.012886492311791476
인덱스는:737
966 405 0.11044282141163697
인덱스는:738
964 415 0

In [8]:
switch_handle(1)
Ac5_payoptions(2,False,2,2)

테스트환경이다.
인덱스는:start
698 429 0.01564084812953819
인덱스는:128
698 429 0.013685625606223804
인덱스는:129
698 429 0.024438300920353914
인덱스는:130
1003 494 2.588807360537452
인덱스는:131
1003 494 0.01857356540738766
인덱스는:132
1003 494 0.014663819403491568
인덱스는:133
1003 494 0.014671275859306852
인덱스는:134
1003 494 0.021497195129712248
인덱스는:135
1003 488 0.31183970085047297
인덱스는:136
1003 488 0.029336493348263788
인덱스는:137
903 511 0.3831930897036262
인덱스는:138
901 512 0.029325308664540858
인덱스는:139
901 512 0.03519260733419362
인덱스는:140
625 385 1.5430275487172087
인덱스는:141
627 391 0.027371484226691837
인덱스는:142
628 393 0.023461272194307287
인덱스는:143
746 416 0.46825680367289135
인덱스는:144
748 417 0.017587216111571928
인덱스는:145
749 417 0.01760492519413323
인덱스는:146
751 416 0.02052855491645782
인덱스는:147
771 414 0.41545228071723767
인덱스는:148
771 414 0.016618808912561726
인덱스는:149
771 414 0.012707664823200269
인덱스는:150
772 414 0.012708829894421406
인덱스는:151
772 414 0.013687489720177625
인덱스는:152
774 419 0.029332765120356142
인덱스는:153


In [9]:
Ac6_payways(False)

In [10]:
Ac7_lastoption(False,"")

인덱스는:start
1174 357 0.015659515643437982
인덱스는:475
1173 358 0.017897617741325273
인덱스는:476
1173 359 0.022267702728741606
인덱스는:477
1172 351 0.3324216197563361
인덱스는:478
1172 351 0.04002773878284865


In [ ]:
review_lists=[]
switch_handle(1)
elements=driver.find_elements(By.CSS_SELECTOR,".sdp-review__article__list__review__content")
for review in elements:
    Temporary_list=[]
    index=elements.index(review)+1
    title=f"{index}번째 문구입니다."
    Temporary_list.append(title)
    Temporary_list.append(review.text)
    review_lists.append(Temporary_list)

# print(review_lists)
for re in review_lists:
    print(re[1])


In [ ]:
review_lists=[]
switch_handle(1)
elements=driver.find_elements(By.CSS_SELECTOR,".sdp-review__article__list__review__content")
for nums in range(2):
    review
    Temporary_list=[]
    index=elements.index(review)+1
    title=f"{index}번째 문구입니다."
    Temporary_list.append(title)
    Temporary_list.append(review.text)
    review_lists.append(Temporary_list)

print(review_lists)
